### Notebook: mcmc_k_equals_c_beta_over_alpha.ipynb

**Objective:** Test the relationship k = c * (β/α) using MCMC on MIDIS observational data, where k is the cosmological scaling parameter and β/α is the laboratory-measured ratio.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import emcee
from scipy.optimize import minimize

# parameters (papermill-friendly)
DATA_CSV = "../data/midis_f560w_masslim.csv"

# Load MIDIS data from SST source
midis_data = pd.read_csv(DATA_CSV)
print("MIDIS observational data:")
print(midis_data)

# Extract data
z = midis_data['z'].values
g = midis_data['g'].values
g_err = midis_data['g_err'].values

# Define log-likelihood
def log_likelihood(params, z, g, g_err):
    ln_g0, k = params
    model = ln_g0 - k * z
    ln_g = np.log(g)
    sigma2 = (g_err / g)**2
    return -0.5 * np.sum((ln_g - model)**2 / sigma2 + np.log(2 * np.pi * sigma2))

# Initial guess from least squares
X = np.column_stack([np.ones_like(z), -z])
ln_g = np.log(g)
weights = 1.0 / (g_err / g)**2
W = np.diag(weights)
beta = np.linalg.inv(X.T @ W @ X) @ (X.T @ W @ ln_g)
ln_g0_init, k_init = beta

print(f"Initial estimates: ln_g0 = {ln_g0_init:.3f}, k = {k_init:.3f}")

# MCMC sampling
nwalkers, ndim = 32, 2
p0 = [ln_g0_init, k_init] + 0.1 * np.random.randn(nwalkers, ndim)

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_likelihood, args=(z, g, g_err))
sampler.run_mcmc(p0, 1000, progress=True)

# Results
samples = sampler.get_chain(discard=200, flat=True)
ln_g0_mcmc, k_mcmc = np.mean(samples, axis=0)
ln_g0_err, k_err = np.std(samples, axis=0)

print(f"\nMCMC Results:")
print(f"ln_g0 = {ln_g0_mcmc:.3f} ± {ln_g0_err:.3f}")
print(f"k = {k_mcmc:.3f} ± {k_err:.3f}")

# Compare with theoretical prediction
beta_over_alpha = 0.0503
k_theoretical = beta_over_alpha  # QH framework prediction

print(f"\nTheoretical k = {k_theoretical:.3f} (from β/α)")
agreement_sigma = abs(k_mcmc - k_theoretical) / k_err
print(f"Agreement: {agreement_sigma:.2f}σ")
